In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
df = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/Obesity/train.csv')

In [39]:
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,...,NObeyesdad,Target,Code Gender,Code History,Code FAVC,Code CAEC,Code Smoke,Code SCC,Code CALC,Code MTRANS
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,...,Overweight_Level_II,3,0,1,1,1,0,0,1,3
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,...,Normal_Weight,1,1,1,1,2,0,0,0,4
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,...,Insufficient_Weight,0,1,1,1,1,0,0,0,3
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,...,Obesity_Type_III,6,1,1,1,1,0,0,1,3
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,...,Overweight_Level_II,3,0,1,1,1,0,0,1,3


In [4]:
df.shape

(20758, 18)

In [35]:
for col in ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']:
  print(df[col].unique())

['Male' 'Female']
['yes' 'no']
['yes' 'no']
['Sometimes' 'Frequently' 'no' 'Always']
['no' 'yes']
['no' 'yes']
['Sometimes' 'no' 'Frequently']
['Public_Transportation' 'Automobile' 'Walking' 'Motorbike' 'Bike']


In [38]:
df['Code Gender'] = df['Gender'].map({'Male':0, 'Female':1})
df['Code History'] = df['family_history_with_overweight'].map({'no':0, 'yes':1})
df['Code FAVC'] = df['FAVC'].map({'no':0, 'yes':1})
df['Code CAEC'] = df['CAEC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
df['Code Smoke'] = df['SMOKE'].map({'no':0, 'yes':1})
df['Code SCC'] = df['SCC'].map({'no':0, 'yes':1})
df['Code CALC'] = df['CALC'].map({'no':0, 'Sometimes':1, 'Frequently':2})
df['Code MTRANS'] = df['MTRANS'].map({'Walking':0, 'Bike':1, 'Motorbike':2, 'Public_Transportation':3, 'Automobile':4})

In [5]:
df['NObeyesdad'].value_counts()

Obesity_Type_III       4046
Obesity_Type_II        3248
Normal_Weight          3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level_II    2522
Overweight_Level_I     2427
Name: NObeyesdad, dtype: int64

In [40]:
df.groupby(['NObeyesdad']).mean()

<ipython-input-40-b541ebfba444>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['NObeyesdad']).mean()


,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Target,Code Gender,Code History,Code FAVC,Code CAEC,Code Smoke,Code SCC,Code CALC,Code MTRANS
NObeyesdad,,,,,,,,,,,,,,,,,,
Insufficient_Weight,10388.804598,19.336502,1.683693,49.860773,2.479756,2.919554,1.744163,1.201782,0.802901,0.0,0.642489,0.425684,0.853349,1.504954,0.002774,0.091954,0.653191,3.018629
Normal_Weight,10136.834523,20.801388,1.670744,61.533289,2.372249,2.889552,1.806204,1.189580,0.630216,1.0,0.538611,0.580792,0.839390,1.395847,0.014925,0.051914,0.673589,2.913368
Obesity_Type_I,10170.039519,25.023615,1.693509,92.371026,2.097052,2.454551,2.129783,0.922710,0.739477,4.0,0.435395,0.983505,0.968041,1.060481,0.008247,0.006186,0.548454,3.228179
Obesity_Type_II,10614.992303,27.707128,1.782715,115.995914,2.425227,2.798439,1.985064,1.029579,0.415601,5.0,0.002463,0.998461,0.983374,1.009236,0.035099,0.001232,0.862993,3.283559
Obesity_Type_III,10507.706871,24.126847,1.676007,117.697452,3.000000,2.999950,2.332338,0.549225,0.548849,6.0,0.998764,0.999753,0.999753,1.001236,0.000989,0.000000,1.000000,3.000000
Overweight_Level_I,10271.952204,23.045197,1.686880,74.228266,2.256583,2.528178,2.069366,1.134657,0.557014,2.0,0.440874,0.688504,0.907705,1.071693,0.010301,0.102596,0.914710,3.060569
Overweight_Level_II,10494.728390,26.032084,1.706163,82.085513,2.224499,2.594109,2.004470,1.060895,0.697984,3.0,0.299366,0.923473,0.786281,1.107058,0.009913,0.009516,0.687946,3.205789


In [8]:
df.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [9]:
df['NObeyesdad'].unique()

array(['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight',
       'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I',
       'Obesity_Type_I'], dtype=object)

In [10]:
df['Target'] = df['NObeyesdad'].map({'Overweight_Level_II':3, 'Normal_Weight':1, 'Insufficient_Weight':0,
       'Obesity_Type_III':6, 'Obesity_Type_II':5, 'Overweight_Level_I':2,
       'Obesity_Type_I':4})

In [43]:
features = ['Age', 'Height', 'Weight', 'Code Gender', 'Code History', 'Code CAEC', 'Code CALC',
       'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [44]:
X = df[features]
y = df['Target']

In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

xg = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xg.fit(X_train, y_train)

y_pred = xg.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.896917148362235


In [47]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=200),
    'RandomForest': RandomForestClassifier(n_estimators=100),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [48]:
cv_scores = {}
for model_name, model in models.items():
  scores = cross_val_score(model, X, y, cv=5)
  cv_scores[model_name] = scores
  print(f"{model_name}: Accuracy per fold: {scores}")
  print(f"{model_name}: Average accuracy: {np.mean(scores)}\n")

best_model = max(cv_scores, key=lambda k: np.mean(cv_scores[k]))
print(f"Best model based on average accuracy: {best_model}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression: Accuracy per fold: [0.70496146 0.70905588 0.71290944 0.71091303 0.70922669]
LogisticRegression: Average accuracy: 0.7094133015281969

RandomForest: Accuracy per fold: [0.89234104 0.88896917 0.89836224 0.89713322 0.89159239]
RandomForest: Average accuracy: 0.8936796110601295

XGBoost: Accuracy per fold: [0.90004817 0.89426782 0.90438343 0.90074681 0.89978318]
XGBoost: Average accuracy: 0.8998458829476288

Best model based on average accuracy: XGBoost


In [52]:
df_new = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/Obesity/test.csv')

In [53]:
df_new.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [54]:
df_new.isna().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
dtype: int64

In [55]:
df_new['Code Gender'] = df_new['Gender'].map({'Male':0, 'Female':1})
df_new['Code History'] = df_new['family_history_with_overweight'].map({'no':0, 'yes':1})
df_new['Code FAVC'] = df_new['FAVC'].map({'no':0, 'yes':1})
df_new['Code CAEC'] = df_new['CAEC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
df_new['Code Smoke'] = df_new['SMOKE'].map({'no':0, 'yes':1})
df_new['Code SCC'] = df_new['SCC'].map({'no':0, 'yes':1})
df_new['Code CALC'] = df_new['CALC'].map({'no':0, 'Sometimes':1, 'Frequently':2})
df_new['Code MTRANS'] = df_new['MTRANS'].map({'Walking':0, 'Bike':1, 'Motorbike':2, 'Public_Transportation':3, 'Automobile':4})

In [56]:
X_new = df_new[features]

In [57]:
xg = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [58]:
xg.fit(X,y)

y_new = xg.predict(X_new)
df_new['Target'] = y_new

In [59]:
df_new['Target'] = df_new['Target'].map({3:'Overweight_Level_II', 1:'Normal_Weight', 0:'Insufficient_Weight',
       6:'Obesity_Type_III', 5:'Obesity_Type_II', 2:'Overweight_Level_I',
       4:'Obesity_Type_I'})

In [60]:
res = pd.DataFrame({'id':df_new['id'], 'NObeyesdad':df_new['Target']})

In [61]:
#res.to_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/Obesity/result2.csv', index=False)